<a href="https://colab.research.google.com/github/Ellena-F/Expense-tracker-and-bank-statement-reader/blob/master/Copy_of_read_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# install package
!pip install pdfminer
!pip install PyPDF2
!pip install tabula-py

In [0]:
# import package
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.collocations import *
from nltk.stem import *
from nltk.stem.porter import *
import seaborn as sns #basic charts
import matplotlib.pyplot as plt #plot setup
from wordcloud import WordCloud #word cloud
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import io
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
import tabula


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

Tokenization


In [0]:
#set filepath
filepath4='/content/drive/My Drive/BankStatement/April.pdf'

In [0]:
#calculate page number for pdf file 
reader = PdfFileReader(open(filepath4, mode='rb' ))
n = reader.getNumPages() 


10

In [0]:
#read pdf file into df
rows4 = tabula.read_pdf(filepath4,
                       pages=("2-"+str(n)),
                       stream=True
                      #  ,encoding = 'ISO-8859-1'
                      ,columns=[36.95,92.72,303.9,408.44,515.88]
                       ,area=(180.61,36.95,749.29,574.99)
                       ,multiple_tables=True
                      #  silent=True
                      # #  pandas_options={
                      # #      'header': None}
                       )


In [0]:
#drop column if na>0.05 of the number of rows
#reassign column name
for i in range(len(rows4)-1):
  rows4[i]=rows4[i].dropna(axis='columns',thresh=rows4[i].shape[0]*0.05,how='all')
  rows4[i].columns=['Date',	'Transaction Details',	'Debits/Withdrawals',	'Credits/Deposits',	'Balance']

In [0]:
#concatenate different page of pdf into 1 df
tryframe22=[]
for i in range(len(rows4)-1):
  tryframe22+=[rows4[i]]

trydf22=pd.concat(tryframe22)
trydf22


In [0]:
trydf22=trydf22.reset_index(drop=True)

for i in range(1,len(trydf22)):

  if pd.isnull(trydf22['Debits/Withdrawals'][i])==True and pd.isnull(trydf22['Credits/Deposits'][i])==True:
    m=i-1
    trydf22['Transaction Details'][m]=trydf22['Transaction Details'][m]+' '+trydf22['Transaction Details'][i]


In [0]:
#drop rows if both debit and credit amount are null
trydf22=trydf22.dropna(subset=['Debits/Withdrawals','Credits/Deposits'], how='all')


In [0]:
#fill null value to 0 
trydf22=trydf22.fillna(0)
trydf22

In [0]:
#define function to check unnecessary information
def check_tran (text):
  if text in ['Transaction Total','Transaction Number']:
    return 1
  else:
    return 0 

In [0]:
trydf22['filter'] = trydf22['Transaction Details'].apply(check_tran)
trydf22

In [0]:
trydf22=trydf22[trydf22['filter'] == 0]
trydf22=trydf22.reset_index(drop=True)
trydf22.tail()

In [0]:
#remove , from numeric value on debit column
#convert debit column value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Debits/Withdrawals'])==str:
    trydf22.loc[i,'Debits/Withdrawals']=float(trydf22.loc[i,'Debits/Withdrawals'].replace(',', ''))
  else:
    pass

In [0]:
#remove , from numeric value on credit column
#convert credit column value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Credits/Deposits'])==str:
   trydf22.loc[i,'Credits/Deposits']=float(trydf22.loc[i,'Credits/Deposits'].replace(',', ''))
  else:
    pass

In [0]:
#remove , from numeric value on balance column
#convert credit balance value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Balance'])==str:
    ba=float(trydf22.loc[i,'Balance'].replace(',', ''))
    trydf22.loc[i,'Balance']=ba

  else:
    pass

In [0]:
#remove row if debits value is not numeric

trydf22=trydf22[trydf22['Debits/Withdrawals'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]


In [0]:
#reset index
trydf22=trydf22.reset_index(drop=True)
trydf22

#Categorise transactions

In [0]:
#Main categories and keyword for transaction
deb_keyword=['TRANSFER',  'AIRBNB','COLES','WOOLWORTHS','CHEMIST WAREHOUSE']
###add more keyword for your own trasaction


deb_cat={'Transfer': ['TRANSFER'], 'Travelling': ['AIRBNB'], 
         'Grocery':['COLES','WOOLWORTHS','CHEMIST WAREHOUSE']}
###add more category for your own transacion
cre_keyword=['TRANSFER']
cre_cat={'Transfer': ['TRANSFER']}

In [0]:
#add column for category and keyword
trydf22['keyword']='unknown'
trydf22['Category']='unknown'
trydf22

In [0]:
#Assign key word for debit record
for word in deb_keyword:
  for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Debits/Withdrawals']>0:
      if word in trydf22['Transaction Details'][i].upper():
        trydf22['keyword'][i]=word
      else:
        pass



In [0]:
#Assign key work for credit record
for word in cre_keyword:
  for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Credits/Deposits']>0:
      if word in trydf22['Transaction Details'][i].upper():
        trydf22['keyword'][i]=word
      else:
        pass

trydf22.head()

Assign category

In [0]:

for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Credits/Deposits']>0:

      for key in cre_cat.keys():
        for value in cre_cat[key]:
          if value in trydf22.loc[i]['keyword']:
            trydf22['Category'][i]=key
trydf22.tail()

In [0]:
for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Debits/Withdrawals']>0:

      for key in deb_cat.keys():
        for value in deb_cat[key]:
          if value in trydf22.loc[i]['keyword']:
            trydf22['Category'][i]=key
trydf22.tail()

In [0]:
#assign empty date value to last record date
trydf22['Date']=trydf22['Date'].replace(0, np.nan).ffill()
trydf22.head()


In [0]:
#output file
# trytrydf.tail()
trydf22.to_csv('finalApril_Statement.csv')
# !cp data.csv "drive/My Drive/"
!cp finalApril_Statement.csv "/content/drive/My Drive"